<a href="https://colab.research.google.com/github/RNimantha/recommendation/blob/main/Recommendations_system_for_Amozon_beauty_product_with_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling pac

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Pyspark

In [35]:
!pip install pyspark

Import module

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
import pyspark.sql.functions as f
import mlflow
from pyspark.ml.evaluation import RegressionEvaluator

In [37]:
spark = SparkSession.builder.appName("Recommendation").getOrCreate()

In [38]:
file_path = '/content/drive/MyDrive/Data_project/ratings_Beauty.csv'
df_buauty = spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path) #data frame, header will infer column types from csv
df_buauty.show()

+--------------+----------+------+----------+
|        UserId| ProductId|Rating| Timestamp|
+--------------+----------+------+----------+
|A39HTATAQ9V7YF|0205616461|   5.0|1369699200|
|A3JM6GV9MNOF9X|0558925278|   3.0|1355443200|
|A1Z513UWSAAO0F|0558925278|   5.0|1404691200|
|A1WMRR494NWEWV|0733001998|   4.0|1382572800|
|A3IAAVS479H7M7|0737104473|   1.0|1274227200|
| AKJHHD5VEH7VG|0762451459|   5.0|1404518400|
|A1BG8QW55XHN6U|1304139212|   5.0|1371945600|
|A22VW0P4VZHDE3|1304139220|   5.0|1373068800|
|A3V3RE4132GKRO|130414089X|   5.0|1401840000|
|A327B0I7CYTEJC|130414643X|   4.0|1389052800|
|A1BG8QW55XHN6U|130414643X|   5.0|1372032000|
| AIFAAVTUYHEHB|130414643X|   4.0|1378252800|
| AVOGV98AYOFG2|1304146537|   5.0|1372118400|
|A22VW0P4VZHDE3|130414674X|   5.0|1371686400|
| AVOGV98AYOFG2|1304168522|   5.0|1372118400|
| A6R426V4J7AOM|1304168522|   5.0|1373414400|
|A22VW0P4VZHDE3|1304174778|   5.0|1372896000|
| AKGB62WGF35J8|1304174778|   5.0|1372896000|
|A22VW0P4VZHDE3|1304174867|   5.0|

In [39]:
df_buauty = df_buauty.drop('Timestamp')
df_buauty.show()

+--------------+----------+------+
|        UserId| ProductId|Rating|
+--------------+----------+------+
|A39HTATAQ9V7YF|0205616461|   5.0|
|A3JM6GV9MNOF9X|0558925278|   3.0|
|A1Z513UWSAAO0F|0558925278|   5.0|
|A1WMRR494NWEWV|0733001998|   4.0|
|A3IAAVS479H7M7|0737104473|   1.0|
| AKJHHD5VEH7VG|0762451459|   5.0|
|A1BG8QW55XHN6U|1304139212|   5.0|
|A22VW0P4VZHDE3|1304139220|   5.0|
|A3V3RE4132GKRO|130414089X|   5.0|
|A327B0I7CYTEJC|130414643X|   4.0|
|A1BG8QW55XHN6U|130414643X|   5.0|
| AIFAAVTUYHEHB|130414643X|   4.0|
| AVOGV98AYOFG2|1304146537|   5.0|
|A22VW0P4VZHDE3|130414674X|   5.0|
| AVOGV98AYOFG2|1304168522|   5.0|
| A6R426V4J7AOM|1304168522|   5.0|
|A22VW0P4VZHDE3|1304174778|   5.0|
| AKGB62WGF35J8|1304174778|   5.0|
|A22VW0P4VZHDE3|1304174867|   5.0|
|A1BG8QW55XHN6U|1304174867|   5.0|
+--------------+----------+------+
only showing top 20 rows



In [40]:

na_counts = df_buauty.select([f.sum(f.col(c).isNull().cast("int")).alias(c) for c in df_buauty.columns])
na_counts.show()

+------+---------+------+
|UserId|ProductId|Rating|
+------+---------+------+
|     0|        0|     0|
+------+---------+------+



In [41]:
rows = df_buauty.count()
cols = len(df_buauty.columns)
print(rows,cols)

2023070 3


* let us understand how freaquently user has buy the product

In [42]:
df_beuaty_agg = df_buauty.select('UserId', 'ProductId').groupby('UserId', 'ProductId').agg(count('*').alias('count')).orderBy('UserId')
df_beuaty_agg.show()

+--------------------+----------+-----+
|              UserId| ProductId|count|
+--------------------+----------+-----+
|A00008821J0F472ND...|B007T8XPC6|    1|
|A000186437REL8X2R...|B000JD4N9M|    1|
|A0002574WYJMBWKNCPY8|B00AH4TJNS|    1|
|A00029263J863WSR0...|B00766SGE8|    1|
|A00031961JI1CBNV98TW|B002OO16QC|    1|
|A000325234LCBTFVL...|B006ZN1B1S|    1|
|A0003526DBE3EVU1DXE5|B00F1AITPM|    1|
|A0004690232S2SX49...|B009L0YD6M|    1|
|A000622834IP6B056...|B00AE078TM|    1|
|A0007094NAUEOIOA53YH|B005UFN3VS|    1|
|A00088021N0VU5Z2U...|B003AMEUS8|    1|
|A0010876CNE3ILIM9HV0|B001KYRVSO|    1|
|A0010876CNE3ILIM9HV0|B004MKEMPI|    1|
|A0010876CNE3ILIM9HV0|B005HKST1U|    1|
|A0010876CNE3ILIM9HV0|B0055MYJ0U|    1|
|A0011102257KBXODK...|B009PZGYJM|    1|
|A001114613O3F18Q5...|B007HSGKXY|    1|
|A00116502XCN8ZW09...|B0024WDWU2|    1|
|A00120381FL204MYH...|B006SVCY6I|    1|
|A001235832OWO8HZG...|B001D5CG9E|    1|
+--------------------+----------+-----+
only showing top 20 rows



In [43]:
df_buauty.groupBy('UserID','ProductID').count().orderBy(desc('count')).show()

+--------------+----------+-----+
|        UserID| ProductID|count|
+--------------+----------+-----+
|A1RGXKJ4RFIOCX|1304624544|    1|
|A2H3NOCLLEPOKQ|7535842801|    1|
|A3VUS3HC1CCVOQ|5357955948|    1|
|A3G6XNM240RMWA|7806397051|    1|
|A2LRQ8GV6PZMHX|7806397051|    1|
| AF508G9N8T5OY|9746427962|    1|
|A264BXIV14DIWA|9511181564|    1|
|A2EDV2NPKN69YM|9788080626|    1|
|A3KK59I1TY3MRB|9788071171|    1|
|A2L8AWOIWML9CH|9790772238|    1|
| A2PZOIS5NOHEA|9788071260|    1|
| ACUHBB2EEXH24|9790777450|    1|
| AYB66VHHXKRQQ|9790778872|    1|
| AXQDDGADNZNNG|9790789556|    1|
|A27JHZ3V0NYS2C|9790799365|    1|
|A3N5L80QA6SDZ2|9790795416|    1|
|A15SQDXX0VZPY2|9790801076|    1|
|A2GWFNKXW7REGE|9790801491|    1|
|A12L11Y59Y8Z54|B00004TMFE|    1|
|A29LEQ3L7XN2BF|B00000JGVX|    1|
+--------------+----------+-----+
only showing top 20 rows



* it can be understand that no user has buy same product more than that

In [44]:
rows = df_beuaty_agg.count()
cols = len(df_beuaty_agg.columns)
print(rows,cols)

2023070 3


In [45]:
#we want to decrement our row size, its almost 10 million, lets reduce it inorder to process our data much faster
df_beuaty_agg = df_beuaty_agg.limit(20000)

*  Let's convert the user id and product id columns into unique integers

In [46]:
old_strindexer = [StringIndexer(inputCol = col, outputCol = col + '_index').fit(df_beuaty_agg) for col in list(set(df_beuaty_agg.columns)- set(['count']))]
indexer = [curr_strindexer.setHandleInvalid("keep") for curr_strindexer in old_strindexer]

pipeline = Pipeline(stages = indexer)

data = pipeline.fit(df_beuaty_agg).transform(df_beuaty_agg)
data.show()

+--------------------+----------+-----+------------+---------------+
|              UserId| ProductId|count|UserId_index|ProductId_index|
+--------------------+----------+-----+------------+---------------+
|A00008821J0F472ND...|B007T8XPC6|    1|      3143.0|        11632.0|
|A000186437REL8X2R...|B000JD4N9M|    1|      3144.0|          637.0|
|A0002574WYJMBWKNCPY8|B00AH4TJNS|    1|      3145.0|        12802.0|
|A00029263J863WSR0...|B00766SGE8|    1|      3146.0|        11283.0|
|A00031961JI1CBNV98TW|B002OO16QC|    1|      3147.0|         7532.0|
|A000325234LCBTFVL...|B006ZN1B1S|    1|      3148.0|          973.0|
|A0003526DBE3EVU1DXE5|B00F1AITPM|    1|      3149.0|        13863.0|
|A0004690232S2SX49...|B009L0YD6M|    1|      3150.0|        12489.0|
|A000622834IP6B056...|B00AE078TM|    1|      3151.0|         2626.0|
|A0007094NAUEOIOA53YH|B005UFN3VS|    1|      3152.0|         2331.0|
|A00088021N0VU5Z2U...|B003AMEUS8|    1|      3153.0|          833.0|
|A0010876CNE3ILIM9HV0|B001KYRVSO| 

In [47]:
data = data.select('UserId_index', 'ProductId_index', 'count').orderBy('UserId_index')
data.show()

+------------+---------------+-----+
|UserId_index|ProductId_index|count|
+------------+---------------+-----+
|         0.0|         6230.0|    1|
|         0.0|        14214.0|    1|
|         0.0|           30.0|    1|
|         0.0|         1591.0|    1|
|         0.0|         6237.0|    1|
|         0.0|         1178.0|    1|
|         0.0|         7140.0|    1|
|         0.0|         4193.0|    1|
|         0.0|         1619.0|    1|
|         0.0|         8222.0|    1|
|         0.0|          727.0|    1|
|         0.0|         3101.0|    1|
|         0.0|          337.0|    1|
|         0.0|         1912.0|    1|
|         0.0|         7369.0|    1|
|         0.0|         3224.0|    1|
|         0.0|         1566.0|    1|
|         0.0|         8184.0|    1|
|         0.0|         3522.0|    1|
|         0.0|            8.0|    1|
+------------+---------------+-----+
only showing top 20 rows



In [48]:
(training, test) = data.randomSplit([0.8,0.2])

In [49]:
USERID = "UserId_index"
PRODUCT = "ProductId_index"
COUNT = "count"

als = ALS(maxIter = 5, regParam = 0.01, userCol = USERID, itemCol = PRODUCT, ratingCol = COUNT)
# als.checkpointInterval = 2
print(als)
model = als.fit(training)

predictions = model.transform(test)

ALS_9e8f7da106b3


In [50]:
recs = model.recommendForAllUsers(10)
recs.show()
recs.take(1)

+------------+--------------------+
|UserId_index|     recommendations|
+------------+--------------------+
|           1|[{422, 1.7441589}...|
|           3|[{40, 1.7581091},...|
|           5|[{118, 1.8540515}...|
|           6|[{156, 1.7248875}...|
|           9|[{699, 1.6388798}...|
|          12|[{261, 1.7268442}...|
|          13|[{240, 1.6458278}...|
|          15|[{40, 1.50683}, {...|
|          16|[{307, 1.8055232}...|
|          17|[{162, 1.7711221}...|
|          19|[{77, 1.9945025},...|
|          20|[{532, 1.2710041}...|
|          22|[{307, 1.6536789}...|
|          26|[{19, 1.6271758},...|
|          27|[{397, 1.3349887}...|
|          28|[{230, 1.6412807}...|
|          31|[{73, 1.6659217},...|
|          34|[{127, 1.9366949}...|
|          35|[{55, 1.6822656},...|
|          37|[{134, 1.7664319}...|
+------------+--------------------+
only showing top 20 rows



[Row(UserId_index=1, recommendations=[Row(ProductId_index=422, rating=1.7441588640213013), Row(ProductId_index=206, rating=1.5565356016159058), Row(ProductId_index=143, rating=1.5024867057800293), Row(ProductId_index=313, rating=1.3172123432159424), Row(ProductId_index=487, rating=1.2949793338775635), Row(ProductId_index=26, rating=1.2366758584976196), Row(ProductId_index=979, rating=1.2233998775482178), Row(ProductId_index=300, rating=1.2151858806610107), Row(ProductId_index=532, rating=1.2074296474456787), Row(ProductId_index=262, rating=1.1620205640792847)])]

* Try Ml flow model fitting



In [51]:
# Set the MLflow tracking URI
mlflow.set_tracking_uri("file:/content/mlruns")

In [52]:
# Start an MLflow Experiment
mlflow.set_experiment("ALS_Model_Experiment")


2024/03/30 19:12:46 INFO mlflow.tracking.fluent: Experiment with name 'ALS_Model_Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/314239866641208409', creation_time=1711825966494, experiment_id='314239866641208409', last_update_time=1711825966494, lifecycle_stage='active', name='ALS_Model_Experiment', tags={}>

Train and Log Model

In [54]:


# Define model parameters
max_iter = 5
reg_param = 0.01

# Train ALS model
als = ALS(maxIter=max_iter, regParam=reg_param, userCol=USERID, itemCol=PRODUCT, ratingCol=COUNT)
model = als.fit(training)

# Make predictions
predictions = model.transform(test)

# Evaluate the model
evaluator = RegressionEvaluator(metricName="rmse", labelCol=COUNT, predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

# Log parameters, metrics, and model
with mlflow.start_run():
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("reg_param", reg_param)
    mlflow.log_metric("rmse", rmse)
    mlflow.spark.log_model(model, "als_model")


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [55]:

# Display predictions
predictions.show()

+------------+---------------+-----+------------+
|UserId_index|ProductId_index|count|  prediction|
+------------+---------------+-----+------------+
|         0.0|           30.0|    1|  0.23025176|
|         0.0|          337.0|    1|  0.16317019|
|         0.0|          772.0|    1| -0.09078446|
|         0.0|         1619.0|    1|  0.15299177|
|         0.0|         2040.0|    1| -0.04487122|
|         0.0|         2042.0|    1|-0.052421413|
|         0.0|         2202.0|    1|  0.28447604|
|         0.0|         3224.0|    1|         NaN|
|         0.0|         3841.0|    1|         NaN|
|         0.0|         5659.0|    1|         NaN|
|         0.0|         7369.0|    1|         NaN|
|         0.0|         8762.0|    1|         NaN|
|         0.0|         9265.0|    1|         NaN|
|         0.0|        10901.0|    1|         NaN|
|         0.0|        12678.0|    1|         NaN|
|         1.0|         3963.0|    1|         NaN|
|         1.0|         4987.0|    1|         NaN|


In [56]:
recs = model.recommendForAllUsers(10)
recs.show()
recs.take(1)

+------------+--------------------+
|UserId_index|     recommendations|
+------------+--------------------+
|           1|[{422, 1.7441589}...|
|           3|[{40, 1.7581091},...|
|           5|[{118, 1.8540515}...|
|           6|[{156, 1.7248875}...|
|           9|[{699, 1.6388798}...|
|          12|[{261, 1.7268442}...|
|          13|[{240, 1.6458278}...|
|          15|[{40, 1.50683}, {...|
|          16|[{307, 1.8055232}...|
|          17|[{162, 1.7711221}...|
|          19|[{77, 1.9945025},...|
|          20|[{532, 1.2710041}...|
|          22|[{307, 1.6536789}...|
|          26|[{19, 1.6271758},...|
|          27|[{397, 1.3349887}...|
|          28|[{230, 1.6412807}...|
|          31|[{73, 1.6659217},...|
|          34|[{127, 1.9366949}...|
|          35|[{55, 1.6822656},...|
|          37|[{134, 1.7664319}...|
+------------+--------------------+
only showing top 20 rows



[Row(UserId_index=1, recommendations=[Row(ProductId_index=422, rating=1.7441588640213013), Row(ProductId_index=206, rating=1.5565356016159058), Row(ProductId_index=143, rating=1.5024867057800293), Row(ProductId_index=313, rating=1.3172123432159424), Row(ProductId_index=487, rating=1.2949793338775635), Row(ProductId_index=26, rating=1.2366758584976196), Row(ProductId_index=979, rating=1.2233998775482178), Row(ProductId_index=300, rating=1.2151858806610107), Row(ProductId_index=532, rating=1.2074296474456787), Row(ProductId_index=262, rating=1.1620205640792847)])]